In [3]:
import pandas as pd
import numpy as np
import random
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [98]:
POP_SIZE = 30
GENERATIONS = 100
MUTATION_RATE = 0.1

#Generates a new metal, each entry in the list is the composition of a different metal
newMetal = generate_individual()


#List depicting the corresponding element to each composition in the newMetal list
Composition_order =['Fe' , 'C', 'Si' , 'Mn' , 'P', 
                    'S' , 'Cr', 'Mo' , 'Ni', 'Cu', 
                    'V', 'Al', 'Mg', 'Se' ,'Sn',
                    'As', 'Ti', 'Zr', 'Nb', 'N',
                    'Co', 'W', 'Ca', 'B', 'O', 
                    'Ta', 'Ce', 'La', 'Zn', 'Pb']


best_metal = genetic_algorithm()




Gen 0: Best melting point: 96.31 °C
Gen 1: Best melting point: 94.00 °C
Gen 2: Best melting point: 94.49 °C
Gen 3: Best melting point: 94.91 °C
Gen 4: Best melting point: 95.02 °C
Gen 5: Best melting point: 96.22 °C
Gen 6: Best melting point: 96.55 °C
Gen 7: Best melting point: 96.33 °C
Gen 8: Best melting point: 96.64 °C
Gen 9: Best melting point: 96.70 °C
Gen 10: Best melting point: 96.63 °C
Gen 11: Best melting point: 97.10 °C
Gen 12: Best melting point: 97.09 °C
Gen 13: Best melting point: 97.29 °C
Gen 14: Best melting point: 97.35 °C
Gen 15: Best melting point: 97.59 °C
Gen 16: Best melting point: 97.88 °C
Gen 17: Best melting point: 97.88 °C
Gen 18: Best melting point: 97.99 °C
Gen 19: Best melting point: 98.12 °C
Gen 20: Best melting point: 98.15 °C
Gen 21: Best melting point: 98.17 °C
Gen 22: Best melting point: 98.29 °C
Gen 23: Best melting point: 98.33 °C
Gen 24: Best melting point: 98.62 °C
Gen 25: Best melting point: 98.73 °C
Gen 26: Best melting point: 98.66 °C
Gen 27: Bes

In [87]:
import random

ELEMENTS = ['Fe', 'C', 'Si', 'Mn', 'P', 'S', 'Cr', 'Mo', 'Ni', 'Cu', 'V', 'Cu', 'Al', 'Mg', 'Se',
            'Sn', 'As', 'Ti', 'Zr', 'Nb', 'N', 'Co', 'W', 'Ca', 'B', 'O', 'Ta', 'Ce', 'La', 'Zn']

# Bounds for the composition of all elements, iron will be limited from 50-100 percent and all other smaller elements will be limited from
# 0 to 5 percent. These bounds can be adjusted as we see fit
BOUNDS = {el: (0, 5) for el in ELEMENTS}
BOUNDS['Fe'] = (50, 100)  # Fe is the base

def generate_individual():
    indiv = {}
    alloy_elements = [el for el in ELEMENTS if el != BASE_ELEMENT]

    # Select the Iron concentraion in between the given bounds
    fe_min, fe_max = BOUNDS[BASE_ELEMENT]
    fe_content = random.uniform(fe_min, fe_max)
    indiv[BASE_ELEMENT] = fe_content

    # Find the remaining content left in the metal that has to be filled with the balance elements
    remaining = 100.0 - fe_content

    # Generate compositions of the remaining balance elements
    raw_values = {}
    for el in alloy_elements:
        low, high = BOUNDS[el]
        raw_values[el] = random.uniform(low, high)

    raw_total = sum(raw_values.values())

    Scale all alloying elements to fit in remaining 
    if raw_total == 0:
        # fallback: set all others to zero if total is 0
        for el in alloy_elements:
            indiv[el] = 0.0
    else:
        scale = remaining / raw_total
        for el in alloy_elements:
            indiv[el] = raw_values[el] * scale

    # Return element 
    return [indiv[el] for el in ELEMENTS]

In [97]:
#Creates a score for the given metal, this will be replaced with the machine learning model
def mock_melting_point(metal):
    score = metal[0]
    return score
    


In [94]:
#mutates the composition of one of the alloys
def mutate(indiv, rate=MUTATION_RATE):
    #picks a random element to vary 
    idx = random.randint(0, len(ELEMENTS) - 1)
    el = ELEMENTS[idx]
    low, high = BOUNDS[el]

    # mutate one element by the specified mutation rate
    delta = random.uniform(-rate, rate) * (high - low)
    new_val = max(low, min(high, indiv[idx] + delta))
    indiv[idx] = new_val

    #normalize the concentration so that it still sums up to be 100%
    total = sum(indiv)
    indiv = [(v / total) * 100.0 for v in indiv]

    # Ensure Fe is within bounds, else redo mutation
    fe_idx = ELEMENTS.index(BASE_ELEMENT)
    if not (BOUNDS['Fe'][0] <= indiv[fe_idx] <= BOUNDS['Fe'][1]):
        return mutate(indiv, rate)  # retry
    return indiv


def crossover(parent1, parent2):
    #picks a random point between the second and second to last index to prevent trivial crossover
    point = random.randint(1, len(ELEMENTS) - 2)
    
    #combines both parents to create the child
    child = parent1[:point] + parent2[point:]

    #Scaling to ensure concentration does not exceed 1
    total = sum(child)
    child = [(v / total) * 100.0 for v in child]

    # Fix iron bounds, if it outside of the bounds, the child is rejected and a new child will be created
    fe_idx = ELEMENTS.index(BASE_ELEMENT)
    if not (BOUNDS['Fe'][0] <= child[fe_idx] <= BOUNDS['Fe'][1]):
        return generate_individual()
    return child

#Use of tournament selection, k individuals are randomly picked from the population and paried with the fitness score "score#
#Only the highest scoring individual is seleted and returned
def select(population, scores, k=3):
    selected = random.choices(list(zip(population, scores)), k=k)
    return max(selected, key=lambda x: x[1])[0]

def genetic_algorithm():
    #Creates an initial population of alloys based on the size specified in POP_SIZE
    population = [generate_individual() for _ in range(POP_SIZE)]

    #Repeat the evolutionary loop over a specified number of generations
    for gen in range(GENERATIONS):
        
        #fitness score calculation for each indivitual in a population
        scores = [mock_melting_point(indiv) for indiv in population]
        new_population = []

        #Creating a new population with mutated child enties
        for _ in range(POP_SIZE):
            p1 = select(population, scores)
            p2 = select(population, scores)
            child = crossover(p1, p2)
            child = mutate(child)
            new_population.append(child)
            
        #Replace the current population with the new population
        population = new_population
        #print the best score for the current population
        best_score = max(scores)
        best_indiv = population[scores.index(best_score)]
        print(f"Gen {gen}: Best melting point: {best_score:.2f} °C")
    
    #Re-evaluate the final population to find the overall best individual 
    final_scores = [mock_melting_point(indiv) for indiv in population]
    best_idx = final_scores.index(max(final_scores))
    best_indiv = population[best_idx]
    print("Best composition found:")
    for el, val in zip(ELEMENTS, best_indiv):
        print(f"  {el}: {val:.2f}%")
    return best_indiv